In [1]:
import multiprocessing

multiprocessing.cpu_count()

72

In [2]:
import os
os.cpu_count()

72

In [3]:
import os
len(os.sched_getaffinity(0))

2

In [2]:
import numpy as np
ddd = np.load('testeps/neighborsSorted.npy')

In [7]:
ddd.shape

(1395712,)

In [15]:
print(ddd[1231000:1232000])

[0.02537813 0.02538194 0.02538194 0.02538455 0.02538455 0.02538572
 0.02538572 0.0253871  0.0253871  0.02539073 0.02539073 0.02539289
 0.02539289 0.02539413 0.02539413 0.02539472 0.02539472 0.02539658
 0.02539658 0.0253969  0.0253969  0.02539859 0.02539859 0.02539984
 0.02539984 0.02540116 0.02540116 0.02540514 0.02540514 0.02540719
 0.02540719 0.0254073  0.0254073  0.02540966 0.02540966 0.02541363
 0.02541363 0.02541413 0.02541413 0.02541431 0.02541431 0.02541479
 0.02541479 0.02541703 0.02541703 0.02541781 0.02541781 0.02541822
 0.02541822 0.02542124 0.02542124 0.02542239 0.02542239 0.02542547
 0.02542547 0.02542716 0.02542751 0.02542751 0.02542884 0.02542884
 0.02543022 0.02543022 0.02543068 0.02543068 0.02543103 0.02543103
 0.02543348 0.02543348 0.02543381 0.02543381 0.02543463 0.02543897
 0.02543924 0.02543924 0.0254398  0.0254398  0.02544223 0.02544431
 0.02544431 0.02544759 0.02544759 0.02544826 0.02544826 0.02544959
 0.02544959 0.0254501  0.02545058 0.02545058 0.02545229 0.0254

In [17]:
lbl = np.load('tenmeans.npy')

In [21]:
import collections
count = collections.Counter(lbl)

In [29]:
print(count)
d = count[1]
print(d)

Counter({1: 22091732, 2: 21943840, 3: 14268546, 9: 14188675, 6: 13434275, 5: 13372273, 8: 12870308, 0: 10779745, 4: 3754617, 7: 3747957})
22091732


In [25]:
vir = np.array([])
vir = np.append(vir, 2)

In [26]:
vir

array([2.])

In [27]:
vir = np.append(vir, 51)
vir

array([ 2., 51.])

In [1]:
import cudf
from cuml.cluster import DBSCAN

In [31]:
# Create and populate a GPU DataFrame
gdf_float = cudf.DataFrame()
gdf_float['0'] = [1.0, 2.0, 5.0]
gdf_float['1'] = [4.0, 2.0, 1.0]
gdf_float['2'] = [4.0, 2.0, 1.0]

# Setup and fit clusters
db = DBSCAN(eps=3.0, min_samples=1).fit_predict(gdf_float)

print(db)

0    0
1    0
2    1
dtype: int32


In [35]:
a = 0
for i in range(3):
    a += db[i]
    
a

1

In [8]:
import astropy

ModuleNotFoundError: No module named 'astropy'

In [2]:
150*150

22500

In [5]:
47*22500

1057500

In [2]:
import numpy as np
import astropy.io.fits
import random
import os
import os.path
from collections import Counter
from PIL import Image
from sklearn.cluster import KMeans
from astropy.time import Time
import csv
from statistics import stdev
from statistics import mean
import math
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import cudf
from cuml.cluster import DBSCAN
from cuml.cluster import KMeans as cudkmean
import timeit



# Normalizing between [1,-1]
def normal(a):
    
    aa = a - a.min()
    bb = a.max() - a.min()
    cc = (2 * aa) / bb
    dd = cc - 1
    return dd


# Unnormal

def unnormal(norm, pure):
    ee = norm + 1
    ff = ee/2
    gg = ff * (pure.max() - pure.min())
    hh = gg + pure.min()
    return hh


# Getting Features

def getFeatures(Name):
    fileData = astropy.io.fits.open(Name)
    B = fileData[1].data
    B = np.array(B, dtype=np.float64)
    Binc = fileData[2].data
    Binc = np.array(Binc, dtype=np.float64)
    Bazi = fileData[3].data
    Bazi = np.array(Bazi, dtype=np.float64)
    DopF = fileData[4].data
    DopF = np.array(DopF, dtype=np.float64)
    #DopS = fileData[5].data
    #DopS = np.array(DopS, dtype=np.float64)
    SLFF = fileData[12].data
    SLFF = np.array(SLFF, dtype=np.float64)
    CI = fileData[32].data
    CI = np.array(CI, dtype=np.float64)


    return B, Binc, Bazi, DopF, SLFF, CI


# Removing that effect due to doppler shifts

def canNoise(Doppler):
    
    y = np.mean(Doppler, 0)
    x = np.array(range(Doppler.shape[1]))
    m, b = np.polyfit(x, y, 1)
    v = m*x + b
    out = Doppler - v

    return out

# Normalizing angles

def normangle(feature):
    
    
    a = np.cos(np.deg2rad(feature))
    
    return a
      

# Making a list for dimensions of observations
def dimlist(filenames):
    
    dimList = list()


    for i in NewList:

        file = astropy.io.fits.open(i)
        x, y = file[1].data.shape
        dimList.append((x, y))
    
    return dimList


# Concatenating all the observations

def concatfeatures(NewList):

    Btot, Binctot, DopFtot, SLFFtot, CItot = np.array([]), np.array([]),np.array([]), np.array([]),np.array([])



    for j in NewList:

        B, Binc, Bazi, DopF, SLFF, CI = getFeatures(j)
        Btot = np.concatenate((Btot, B), axis=None)
        Binctot = np.concatenate((Binctot, Binc), axis=None)
        DopFtot = np.concatenate((DopFtot, canNoise(DopF)), axis=None)
        SLFFtot = np.concatenate((SLFFtot, SLFF), axis=None)
        CItot = np.concatenate((CItot, CI), axis=None)
    
    
    return Btot, Binctot, DopFtot, SLFFtot, CItot

#Concateraw
def conpure(NewList):

    Btot, DopFtot, SLFFtot, CItot = np.array([]),np.array([]), np.array([]),np.array([])

    for j in NewList:

        B, Binc, Bazi, DopF, SLFF, CI = getFeatures(j)
        Btot = np.concatenate((Btot, B), axis=None)
        DopFtot = np.concatenate((DopFtot, DopF), axis=None)
        SLFFtot = np.concatenate((SLFFtot, SLFF), axis=None)
        CItot = np.concatenate((CItot, CI), axis=None)
    
    
    return Btot, DopFtot, SLFFtot, CItot


# Plot mappings
def pltmaps(labels, dimList, k):
    
    start = 0
    end = 0

    for s, tple in enumerate(dimList):

        end += tple[0] * tple[1]

        pltimg(labels[start:end], tple, s)

        start = end 
        
def pltimg(labels, tple, s):
    
    fileData = (NewList[s])
    v = fileData[42:-5]
    labless = labels.reshape(tple)
    plt.figure(figsize=(16,9))
    plt.imshow(labless, cmap="plasma", origin="bottom")
    plt.colorbar()
    plt.savefig(dir+'/'+ v + '.png')
    plt.close()
    

    
    
    
NorthDir='/glade/work/egeland/hinode_synoptic/north'
allfiles = [os.path.join(NorthDir, f) for f in sorted(os.listdir(NorthDir))]

# Removing all sav files
NewList=[item for i,item in enumerate(allfiles) if i%2==0]


b = []

for item in NewList:
    
    fileData = astropy.io.fits.open(item)
    a = fileData[0].header['YSCALE']
    if a > 0.16 :
        b.append(item)

NewList = []       
NewList = b

print(len(NewList))


dimList = dimlist(NewList)

Btot, Binctot, DopFtot, SLFFtot, CItot = concatfeatures(NewList)

for it, item in enumerate(Btot):
    
    if item < 200:
        
        Btot[it] = 0
        Binctot[it] = 90
        SLFFtot[it] = 0
        
    elif item > 2000:
        
        Btot[it] = 2000
print(len(Btot))



# Normalizing Features
Btotn = normal(Btot)
Binctotn = normangle(Binctot)
DopFtotn = normal(DopFtot)
SLFFtotn = normal(SLFFtot)
CItotn = normal(CItot)

da = np.array([Btotn, Binctotn, DopFtotn, SLFFtotn, CItotn])
fda = np.transpose(da)

47
22620160


In [7]:
fdda = fda[:1000000,]
X_df = pd.DataFrame({'fea%d'%i: fda[:1000000, i] for i in range(fda.shape[1])})
cudf.set_allocator("managed")
X_gpu = cudf.DataFrame.from_pandas(X_df)
start = timeit.default_timer()
db = DBSCAN(eps=0.4, min_samples=1000).fit(X_df)
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  850.0793396048248


In [8]:
start = timeit.default_timer()
cluster_labels = db.labels_
silhouette_avg = silhouette_score(fdda, cluster_labels)
print(silhouette_avg)
stop = timeit.default_timer()
print('Time: ', stop - start)

0.8154241498066191
Time:  20230.682939294726
